In [1]:
import pandas as pd
import numpy as np

In [2]:
klienci = pd.read_csv('Dane/KLIENCI.csv', encoding='charmap')
klienci = klienci[klienci['ADRES_KRAJ'] == "Polska"]
klienci = klienci[klienci['RODZAJ_KLIENTA'] == "O"]
klienci = klienci[['ID_KLIENTA', 'DATA_URODZENIA', 'PLEC', 'STAN_CYWILNY', 'WYKSZTALCENIE']]
klienci['DATA_URODZENIA'] = klienci['DATA_URODZENIA'].apply(lambda x: np.int(x[:2]) if type(x) == str else 0)
klienci = klienci.set_index('ID_KLIENTA')
klienci

,DATA_URODZENIA,PLEC,STAN_CYWILNY,WYKSZTALCENIE
ID_KLIENTA,,,,
331612008,86,M,NaN,NaN
615222016,88,M,NaN,NaN
149672001,35,K,¯onaty / mê¿atka,NaN
311042007,65,K,¯onaty / mê¿atka,NaN
165872001,51,M,NaN,NaN
...,...,...,...,...
313962007,83,M,NaN,NaN
316442007,64,K,¯onaty / mê¿atka,NaN
334612009,61,K,NaN,NaN


In [3]:
stany_konta = pd.read_csv('Dane/HACK_UMOWY_STANY.csv', delimiter=";")[['ID_KLIENTA', 'DATA_DANYCH', 'SRODKI_KLIENTA']]

In [4]:
sredni_stan_konta = stany_konta.groupby(['ID_KLIENTA', 'DATA_DANYCH']).sum().groupby('ID_KLIENTA').mean()
sredni_stan_konta

,SRODKI_KLIENTA
ID_KLIENTA,
51999,644451.212121
71999,84010.687297
81999,27605.980000
91999,6269.897742
121999,11820.060833
...,...
675802019,0.000000
675812019,0.000000
675862019,0.000000


In [5]:
karty = pd.read_csv('Dane/KARTY.csv')[['ID_UMOWY', 'LICZBA_KART_AKTYWNYCH']]

In [6]:
karty = karty.groupby('ID_UMOWY').sum()
karty

,LICZBA_KART_AKTYWNYCH
ID_UMOWY,
17722005,3
17752005,1
17762005,2
17912005,1
17992005,1
...,...
1698412019,1
1698492019,1
1698522019,1


In [7]:
konta = pd.read_csv('Dane/RACHUNKI.csv')[['ID_UMOWY', 'ID_KLIENTA', 'KOD_PRODUKTU']]
konta

,ID_UMOWY,ID_KLIENTA,KOD_PRODUKTU
0,22182005,230562002,WRR*
1,110612005,125602001,ROR*
2,77212005,175332001,WRR*
3,27782005,57471999,WRR*
4,147032005,188772001,ROR*
...,...,...,...
37725,1619862019,223342001,RPV
37726,1624002019,664062019,RPV
37727,1595922019,244162003,RPV
37728,1608462019,274602006,RPV


In [8]:
kanaly = pd.read_csv('Dane/KANALY_ELEKTR.csv')
kanaly['INTERNET'] = kanaly['KANAL'] == 'internet'
kanaly['SMS'] = kanaly['KANAL'] == 'SMS'
kanaly = kanaly.drop('KANAL', axis=1)
kanaly = kanaly.groupby('ID_KLIENTA').any()
kanaly

,INTERNET,SMS
ID_KLIENTA,,
121999,True,False
131999,True,False
141999,True,False
241999,True,False
351999,True,False
...,...,...
680572019,True,False
680582019,True,False
680622019,True,False


In [9]:
kredyty = pd.read_csv('Dane/HACK_KREDYTY.csv', delimiter=';')[['ID_KLIENTA', 'GRUPA_RYZYKA', 'KWOTA', 'STATUS']]
# wszystkie sa PLN
kredyty = kredyty[kredyty['STATUS'] == 'czynny'].drop('STATUS', axis=1)
# niektorzy maja dwie grupy ryzyka, ale ignoruje ten przypadek
kredyty = pd.concat([kredyty.groupby('ID_KLIENTA')['KWOTA'].sum(), kredyty.groupby('ID_KLIENTA')['GRUPA_RYZYKA'].first()], axis=1)
kredyty

,KWOTA,GRUPA_RYZYKA
ID_KLIENTA,,
1621999,2700.0,N
3821999,3000.0,N
4341999,500.0,N
4561999,5100.0,N
4891999,229887.0,N
...,...,...
680322019,150000.0,N
680342019,20000.0,N
680392019,23000.0,N


In [10]:
dane_operacje = pd.read_csv('Dane/dane_hack_operacje_sredniki.csv', sep=';', error_bad_lines=False, header=None)
columns = ["DATA_OPERACJI", "ID_UMOWY", "KANAL", "KOD_OPERACJI", "KWOTA_OBCIAZENIA", "KWOTA_UZNANIA", "TYTUL",
           "BENEFICJENT", "ZLECENIODAWCA", "KURS_OBCIAZENIA", "KURS_UZNANIA", "KATEGORIA_WG_TYTULU", "NADKATEGORIA"]
dane_operacje.columns = columns

columns_to_drop = ["KANAL", "KOD_OPERACJI", "TYTUL",
           "BENEFICJENT", "ZLECENIODAWCA", "KURS_OBCIAZENIA", "KURS_UZNANIA", "KATEGORIA_WG_TYTULU", "NADKATEGORIA"]
dane_operacje_rachunek = dane_operacje.drop(columns=columns_to_drop)

dane_operacje_rachunek['DATA_OPERACJI'] = dane_operacje_rachunek['DATA_OPERACJI'].apply(lambda x: str(x)[:7])
srednie_obciazenia_uznania = dane_operacje_rachunek.groupby(['ID_UMOWY', 'DATA_OPERACJI']).sum().groupby('ID_UMOWY').mean()
srednie_obciazenia_uznania

b'Skipping line 1883228: expected 13 fields, saw 14\nSkipping line 1891001: expected 13 fields, saw 14\nSkipping line 1893533: expected 13 fields, saw 14\nSkipping line 1894931: expected 13 fields, saw 14\nSkipping line 1897354: expected 13 fields, saw 14\nSkipping line 1899807: expected 13 fields, saw 14\n'
b'Skipping line 1939106: expected 13 fields, saw 14\nSkipping line 1939141: expected 13 fields, saw 14\nSkipping line 1939690: expected 13 fields, saw 14\nSkipping line 1939923: expected 13 fields, saw 14\nSkipping line 1940483: expected 13 fields, saw 14\nSkipping line 1940514: expected 13 fields, saw 14\nSkipping line 1940617: expected 13 fields, saw 14\nSkipping line 1941131: expected 13 fields, saw 14\nSkipping line 1942852: expected 13 fields, saw 14\nSkipping line 1943149: expected 13 fields, saw 14\nSkipping line 1943188: expected 13 fields, saw 14\nSkipping line 1943325: expected 13 fields, saw 14\nSkipping line 1944170: expected 13 fields, saw 14\nSkipping line 1944302: ex

b'Skipping line 2425415: expected 13 fields, saw 14\nSkipping line 2425533: expected 13 fields, saw 14\nSkipping line 2425625: expected 13 fields, saw 14\nSkipping line 2425776: expected 13 fields, saw 14\nSkipping line 2426153: expected 13 fields, saw 14\nSkipping line 2426162: expected 13 fields, saw 14\nSkipping line 2426540: expected 13 fields, saw 14\nSkipping line 2426746: expected 13 fields, saw 14\nSkipping line 2427008: expected 13 fields, saw 14\nSkipping line 2428897: expected 13 fields, saw 14\nSkipping line 2429430: expected 13 fields, saw 14\nSkipping line 2429801: expected 13 fields, saw 14\nSkipping line 2430011: expected 13 fields, saw 14\nSkipping line 2430143: expected 13 fields, saw 14\nSkipping line 2430642: expected 13 fields, saw 14\nSkipping line 2430766: expected 13 fields, saw 14\nSkipping line 2430884: expected 13 fields, saw 14\nSkipping line 2431093: expected 13 fields, saw 14\nSkipping line 2431232: expected 13 fields, saw 14\nSkipping line 2431439: expect

b'Skipping line 2687921: expected 13 fields, saw 14\nSkipping line 2688005: expected 13 fields, saw 14\nSkipping line 2688218: expected 13 fields, saw 14\nSkipping line 2688296: expected 13 fields, saw 14\nSkipping line 2689265: expected 13 fields, saw 14\nSkipping line 2690205: expected 13 fields, saw 14\nSkipping line 2690561: expected 13 fields, saw 14\nSkipping line 2690625: expected 13 fields, saw 14\nSkipping line 2690805: expected 13 fields, saw 14\nSkipping line 2691543: expected 13 fields, saw 14\nSkipping line 2692460: expected 13 fields, saw 14\nSkipping line 2693298: expected 13 fields, saw 14\nSkipping line 2693546: expected 13 fields, saw 14\nSkipping line 2693732: expected 13 fields, saw 14\nSkipping line 2694102: expected 13 fields, saw 14\nSkipping line 2694672: expected 13 fields, saw 14\nSkipping line 2694985: expected 13 fields, saw 14\nSkipping line 2695472: expected 13 fields, saw 14\nSkipping line 2696212: expected 13 fields, saw 14\nSkipping line 2696500: expect

b'Skipping line 2950622: expected 13 fields, saw 14\nSkipping line 2950999: expected 13 fields, saw 14\nSkipping line 2951784: expected 13 fields, saw 14\nSkipping line 2952285: expected 13 fields, saw 14\nSkipping line 2952546: expected 13 fields, saw 14\nSkipping line 2952789: expected 13 fields, saw 14\nSkipping line 2953798: expected 13 fields, saw 14\nSkipping line 2953921: expected 13 fields, saw 14\nSkipping line 2954437: expected 13 fields, saw 14\nSkipping line 2955910: expected 13 fields, saw 14\nSkipping line 2956110: expected 13 fields, saw 14\nSkipping line 2956577: expected 13 fields, saw 14\nSkipping line 2956828: expected 13 fields, saw 14\nSkipping line 2956995: expected 13 fields, saw 14\nSkipping line 2957571: expected 13 fields, saw 14\nSkipping line 2957651: expected 13 fields, saw 14\nSkipping line 2957665: expected 13 fields, saw 14\nSkipping line 2958307: expected 13 fields, saw 14\nSkipping line 2958434: expected 13 fields, saw 14\nSkipping line 2958634: expect

b'Skipping line 3216702: expected 13 fields, saw 14\nSkipping line 3217415: expected 13 fields, saw 14\nSkipping line 3218481: expected 13 fields, saw 14\nSkipping line 3218577: expected 13 fields, saw 14\nSkipping line 3223221: expected 13 fields, saw 14\nSkipping line 3224296: expected 13 fields, saw 14\nSkipping line 3225639: expected 13 fields, saw 14\nSkipping line 3226558: expected 13 fields, saw 14\nSkipping line 3226762: expected 13 fields, saw 14\nSkipping line 3227457: expected 13 fields, saw 14\nSkipping line 3227609: expected 13 fields, saw 14\nSkipping line 3227893: expected 13 fields, saw 14\nSkipping line 3228931: expected 13 fields, saw 14\nSkipping line 3229922: expected 13 fields, saw 14\nSkipping line 3229988: expected 13 fields, saw 14\nSkipping line 3230287: expected 13 fields, saw 14\nSkipping line 3231840: expected 13 fields, saw 14\nSkipping line 3232017: expected 13 fields, saw 14\nSkipping line 3232143: expected 13 fields, saw 14\nSkipping line 3232152: expect

...

b'Skipping line 11806767: expected 13 fields, saw 14\nSkipping line 11807081: expected 13 fields, saw 14\nSkipping line 11808996: expected 13 fields, saw 14\nSkipping line 11811842: expected 13 fields, saw 14\nSkipping line 11813577: expected 13 fields, saw 14\nSkipping line 11816265: expected 13 fields, saw 14\nSkipping line 11818323: expected 13 fields, saw 14\nSkipping line 11818621: expected 13 fields, saw 14\nSkipping line 11818787: expected 13 fields, saw 14\nSkipping line 11818892: expected 13 fields, saw 14\nSkipping line 11820866: expected 13 fields, saw 14\nSkipping line 11820990: expected 13 fields, saw 14\nSkipping line 11823307: expected 13 fields, saw 14\nSkipping line 11825263: expected 13 fields, saw 14\nSkipping line 11825447: expected 13 fields, saw 14\nSkipping line 11825852: expected 13 fields, saw 14\nSkipping line 11827672: expected 13 fields, saw 14\nSkipping line 11828070: expected 13 fields, saw 14\nSkipping line 11830770: expected 13 fields, saw 14\nSkipping l

b'Skipping line 12070513: expected 13 fields, saw 14\nSkipping line 12070698: expected 13 fields, saw 14\nSkipping line 12072941: expected 13 fields, saw 14\nSkipping line 12075646: expected 13 fields, saw 14\nSkipping line 12077977: expected 13 fields, saw 14\nSkipping line 12080176: expected 13 fields, saw 14\nSkipping line 12080205: expected 13 fields, saw 14\nSkipping line 12080475: expected 13 fields, saw 14\nSkipping line 12082331: expected 13 fields, saw 14\nSkipping line 12082775: expected 13 fields, saw 14\nSkipping line 12085140: expected 13 fields, saw 14\nSkipping line 12087495: expected 13 fields, saw 14\nSkipping line 12089507: expected 13 fields, saw 14\nSkipping line 12089559: expected 13 fields, saw 14\nSkipping line 12089572: expected 13 fields, saw 14\nSkipping line 12089697: expected 13 fields, saw 14\nSkipping line 12089821: expected 13 fields, saw 14\nSkipping line 12094072: expected 13 fields, saw 14\nSkipping line 12094268: expected 13 fields, saw 14\nSkipping l

b'Skipping line 12331753: expected 13 fields, saw 14\nSkipping line 12331887: expected 13 fields, saw 14\nSkipping line 12332299: expected 13 fields, saw 14\nSkipping line 12332630: expected 13 fields, saw 14\nSkipping line 12333916: expected 13 fields, saw 14\nSkipping line 12333926: expected 13 fields, saw 14\nSkipping line 12334946: expected 13 fields, saw 14\nSkipping line 12336432: expected 13 fields, saw 14\nSkipping line 12336795: expected 13 fields, saw 14\nSkipping line 12336844: expected 13 fields, saw 14\nSkipping line 12336874: expected 13 fields, saw 14\nSkipping line 12336919: expected 13 fields, saw 14\nSkipping line 12336997: expected 13 fields, saw 14\nSkipping line 12339103: expected 13 fields, saw 14\nSkipping line 12341522: expected 13 fields, saw 14\nSkipping line 12342069: expected 13 fields, saw 14\nSkipping line 12342236: expected 13 fields, saw 14\nSkipping line 12344793: expected 13 fields, saw 14\nSkipping line 12345030: expected 13 fields, saw 14\nSkipping l

b'Skipping line 13116856: expected 13 fields, saw 14\nSkipping line 13116962: expected 13 fields, saw 14\nSkipping line 13118571: expected 13 fields, saw 14\nSkipping line 13118910: expected 13 fields, saw 14\nSkipping line 13119385: expected 13 fields, saw 14\nSkipping line 13119410: expected 13 fields, saw 14\nSkipping line 13120371: expected 13 fields, saw 14\nSkipping line 13120624: expected 13 fields, saw 14\nSkipping line 13120919: expected 13 fields, saw 14\nSkipping line 13121709: expected 13 fields, saw 14\nSkipping line 13121809: expected 13 fields, saw 14\nSkipping line 13122264: expected 13 fields, saw 14\nSkipping line 13122953: expected 13 fields, saw 14\nSkipping line 13123983: expected 13 fields, saw 14\nSkipping line 13124422: expected 13 fields, saw 14\nSkipping line 13125301: expected 13 fields, saw 14\nSkipping line 13125535: expected 13 fields, saw 14\nSkipping line 13125540: expected 13 fields, saw 14\nSkipping line 13125631: expected 13 fields, saw 14\nSkipping l

,KWOTA_OBCIAZENIA,KWOTA_UZNANIA
ID_UMOWY,,
1.820050e+05,276.397857,7.567857e+00
2.220050e+05,402.067143,7.247143e+00
5.220050e+05,87.400222,3.038867e+01
1.992005e+06,654.920000,3.595000e+01
2.412005e+06,1227.755000,1.240250e+01
...,...,...
1.702592e+09,0.000000,5.600000e+02
1.702602e+09,0.000000,2.552800e+06
1.702612e+09,0.000000,1.000000e+01


In [11]:
obciazenia = konta.merge(srednie_obciazenia_uznania, left_on='ID_UMOWY', right_index=True)[['ID_KLIENTA', 'KWOTA_OBCIAZENIA', 'KWOTA_UZNANIA']].groupby('ID_KLIENTA').sum()

In [12]:
from collections import Counter
ilosc_kont = konta[['ID_KLIENTA', 'KOD_PRODUKTU']].groupby('ID_KLIENTA')['KOD_PRODUKTU'].value_counts().unstack(level=-1)[['ROR*', 'RPV', 'WRR*', 'AVIS', 'ZAWS', 'WOF', 'AVS', 'STAR', 'ZASE']].fillna(0)
ilosc_kont

KOD_PRODUKTU,ROR*,RPV,WRR*,AVIS,ZAWS,WOF,AVS,STAR,ZASE
ID_KLIENTA,,,,,,,,,
51999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71999,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91999,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121999,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
131999,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
681262019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
681282019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
681342019,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [13]:
aktywne_karty = konta.merge(karty, left_on='ID_UMOWY', right_index=True).groupby('ID_KLIENTA')['LICZBA_KART_AKTYWNYCH'].sum()
aktywne_karty

ID_KLIENTA
241999       1
1041999      1
1461999      2
1601999      1
2121999      1
            ..
680022019    1
680052019    1
680072019    1
680102019    1
680112019    1
Name: LICZBA_KART_AKTYWNYCH, Length: 11720, dtype: int64

In [14]:
data = klienci.join(sredni_stan_konta)
data = data.join(kredyty)
data = data.join(kanaly)
data = data.join(obciazenia)
data = data.join(ilosc_kont)
data = data.join(aktywne_karty)
data

,DATA_URODZENIA,PLEC,STAN_CYWILNY,WYKSZTALCENIE,SRODKI_KLIENTA,KWOTA,GRUPA_RYZYKA,INTERNET,SMS,KWOTA_OBCIAZENIA,...,ROR*,RPV,WRR*,AVIS,ZAWS,WOF,AVS,STAR,ZASE,LICZBA_KART_AKTYWNYCH
ID_KLIENTA,,,,,,,,,,,,,,,,,,,,,
331612008,86,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
615222016,88,M,NaN,NaN,2029.800870,72000.0,N,True,False,4148.327692,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
149672001,35,K,¯onaty / mê¿atka,NaN,20480.548852,NaN,NaN,NaN,NaN,986.018036,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN
311042007,65,K,¯onaty / mê¿atka,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165872001,51,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313962007,83,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
316442007,64,K,¯onaty / mê¿atka,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
334612009,61,K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
for column in ['PLEC', 'SRODKI_KLIENTA', 'KWOTA', 'INTERNET', 'SMS',
               'KWOTA_OBCIAZENIA', 'KWOTA_UZNANIA', 'ROR*', 'RPV', 'WRR*', 'AVIS',
               'ZAWS', 'WOF', 'AVS', 'STAR', 'ZASE', 'LICZBA_KART_AKTYWNYCH']:
    data[column].fillna(0, inplace=True)

In [16]:
for column in ['INTERNET', 'SMS']:
    data[column] = data[column].astype(np.int)

In [17]:
data = data.drop(columns=['WYKSZTALCENIE'])
data = data.drop(columns=['GRUPA_RYZYKA'])

In [18]:
data['STAN_CYWILNY'].value_counts()

¯onaty / mê¿atka                 9965
Kawaler / pan                    3414
Wdowiec / wdowa                   824
Rozwiedziony / rozwiedzio         154
Pozosta³e                          17
Niezne                             11
Separacja                          10
Kohabitacja (wspólne po¿ycie)       5
Name: STAN_CYWILNY, dtype: int64

In [19]:
data = data.join(pd.get_dummies(data['PLEC'])).drop(columns=['PLEC'])
data = data.join(pd.get_dummies(data['STAN_CYWILNY'])).drop(columns=['STAN_CYWILNY'])

In [20]:
polisy = pd.read_excel('Dane/HACK_POLISY.xlsx')[['ID_KLIENTA', 'RODZAJ_UBEZPIECZENIA']]
rodzaje_polis = polisy.groupby('ID_KLIENTA')['RODZAJ_UBEZPIECZENIA'].apply(lambda x:x.value_counts().index[0])
rodzaje_polis

ID_KLIENTA
51999           1
1621999        13
2211999         1
2391999         3
3309999         2
               ..
678522019      13
678672019      13
678802019      13
679212019       2
74030515878    13
Name: RODZAJ_UBEZPIECZENIA, Length: 4133, dtype: int64

In [21]:
data = data.join(rodzaje_polis).fillna(0)

In [22]:
data.to_csv('Dane/dane_treningowe.csv', sep=';')

In [23]:
data

,DATA_URODZENIA,SRODKI_KLIENTA,KWOTA,INTERNET,SMS,KWOTA_OBCIAZENIA,KWOTA_UZNANIA,ROR*,RPV,WRR*,...,M,Kawaler / pan,Kohabitacja (wspólne po¿ycie),Niezne,Pozosta³e,Rozwiedziony / rozwiedzio,Separacja,Wdowiec / wdowa,¯onaty / mê¿atka,RODZAJ_UBEZPIECZENIA
ID_KLIENTA,,,,,,,,,,,,,,,,,,,,,
331612008,86,0.000000,0.0,0,0,0.000000,0.000000,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0.0
615222016,88,2029.800870,72000.0,1,0,4148.327692,4150.468462,1.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,13.0
149672001,35,20480.548852,0.0,0,0,986.018036,1194.956786,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0.0
311042007,65,0.000000,0.0,0,0,0.000000,0.000000,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0.0
165872001,51,0.000000,0.0,0,0,0.000000,0.000000,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313962007,83,0.000000,0.0,0,0,0.000000,0.000000,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0.0
316442007,64,0.000000,0.0,0,0,0.000000,0.000000,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0.0
334612009,61,0.000000,0.0,0,0,0.000000,0.000000,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.0
